In [2]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import pandas as pd
import torch
import numpy
import re
from sklearn.model_selection import train_test_split
from transformers import AutoModel, AutoTokenizer
from sklearn.svm import SVC
from joblib import dump
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

ModuleNotFoundError: No module named 'torch'

In [7]:
df = pd.read_csv('data_after_preprocessing.csv')
df.head()

,Unnamed: 0,docId,sourceName,text,Nhãn
0,0,a17cf79fc0f434d3a1b5999e9c61bada,hà nội mới,"môi_trường xanh ( hnnn ) - dân than tổ ong , đ...",1
1,1,da9bd20f8be7250fbf0edd9c692a5382,cafebiz,"ngoại_thành đốt rơm rạ , dân nội_đô khổ_sở ô_n...",-1
2,2,66588c6e364078182b9b17deb34e88de,thanh niên,"ngoại_thành đốt rơm rạ ' khổ ' nội_thành , ngư...",-1
3,3,6f39aca6dbfd1769799c523c74e6705e,vĩnh phúc online,rơm rạ thu_hoạch - lợi_ích kép nông_nghiệp tha...,1
4,4,f76249f7a06d1be55ded50e0335a1bfe,báo thái bình,hưng_hà : tăng_cường biện_pháp ngăn_chặn hành_...,1


In [8]:
def load_bert():
    v_phobert = AutoModel.from_pretrained("vinai/phobert-base")
    v_tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)
    return v_phobert, v_tokenizer

In [9]:
def load_stopwords():
    sw = []
    with open("stopwords.txt", encoding='utf-8') as f:
        lines = f.readlines()
    for line in lines:
        sw.append(line.replace("\n",""))
    return sw

In [10]:
def standardize_data(row):
    # Xóa dấu chấm, phẩy, hỏi ở cuối câu
    row = re.sub(r"[\.,\?]+$-", "", row)
    # Xóa tất cả dấu chấm, phẩy, chấm phẩy, chấm thang, ... trong câu
    row = row.replace(",", " ").replace(".", " ") \
        .replace(";", " ").replace("“", " ") \
        .replace(":", " ").replace("”", " ") \
        .replace('"', " ").replace("'", " ") \
        .replace("!", " ").replace("?", " ") \
        .replace("-", " ").replace("?", " ")
    return row

In [11]:
def make_bert_features(v_text):
    global phobert, sw
    v_tokenized = []
    max_len = 256 # Mỗi câu dài tối đa 256 từ
    for i_text in v_text:
        #print("Đang xử lý line = ", i_text)
        # Tokenize bởi BERT
        line = tokenizer.encode(i_text, max_length = 256)
        v_tokenized.append(line)

    # Chèn thêm số 1 vào cuối câu nếu như không đủ 256 từ
    padded = numpy.array([i + [1] * (max_len - len(i)) for i in v_tokenized])
    print('padded:', padded[0])
    print('len padded:', padded.shape)

    # Đánh dấu các từ thêm vào = 0 để không tính vào quá trình lấy features
    attention_mask = numpy.where(padded == 1, 0, 1)
    print('attention mask:', attention_mask[0])

    # Chuyển thành tensor
    padded = torch.tensor(padded).to(torch.long)
    print("Padd = ",padded.size())
    attention_mask = torch.tensor(attention_mask)

    # Lấy features dầu ra từ BERT
    with torch.no_grad():
        last_hidden_states = phobert(input_ids= padded, attention_mask=attention_mask)

    v_features = last_hidden_states[0][:, 0, :].numpy()
    print(v_features.shape)
    return v_features

In [12]:
print("Chuẩn bị nạp model BERT....")
phobert, tokenizer = load_bert()
print("Đã nạp xong model BERT.")

print("Chuẩn bị load dữ liệu....")
text = df['text'].to_list()
label = df['Nhãn'].to_list()
print("Đã load dữ liệu xong")

print("Chuẩn bị tạo features từ BERT.....")
features = make_bert_features(text)
print("Đã tạo xong features từ BERT")

Chuẩn bị nạp model BERT....


Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples t

Đã nạp xong model BERT.
Chuẩn bị load dữ liệu....
Đã load dữ liệu xong
Chuẩn bị tạo features từ BERT.....
padded: [    0   427   656    20 39360 43004    19    31   452  2497  1391  2662
     4  1933 11255 29161  2881     4  2405  2931  2694  6735     4  1100
     4   715   448     5     5     5  1571  1413   143 15363     4  1784
     5   775   589  1961   427     4   390  1571  1413     4   455    18
   452  1243   656     4  1024  1618   114 44068  2151  2709   701   571
     7     9   582     4   260     5     2     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1    

In [13]:

X_train, X_test, Y_train, Y_test = train_test_split(features, label, test_size=0.2, random_state=257)

In [50]:
SVM = SVC(kernel='linear', probability=True, gamma=0.1)
SVM.fit(X_train,Y_train)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(X_test)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Y_test)*100)

SVM Accuracy Score ->  78.19148936170212


In [15]:
print(classification_report(Y_test,predictions_SVM))

              precision    recall  f1-score   support

          -1       0.74      0.69      0.72        75
           1       0.81      0.84      0.82       113

    accuracy                           0.78       188
   macro avg       0.77      0.77      0.77       188
weighted avg       0.78      0.78      0.78       188



In [33]:
param_grid = {  'boosting_type': ['gbdt','goss','dart'],
                'n_estimators': [1000,1100,1200], 
                'learning_rate': [0.05,0.08,0.1,0.2],
                'num_leaves': [15, 20, 31, 40],
                'scale_pos_weight': [1.0,0.8,0.7]
            }

In [41]:
grid = GridSearchCV(LGBMClassifier(),param_grid,refit=True,verbose=1,scoring = 'roc_auc')
grid.fit(X_train,Y_train)

Fitting 5 folds for each of 720 candidates, totalling 3600 fits


GridSearchCV(estimator=LGBMClassifier(),
             param_grid={'boosting_type': ['gbdt', 'goss', 'dart'],
                         'learning_rate': [0.01, 0.05, 0.08, 0.1, 0.2],
                         'n_estimators': [100, 200, 500, 1000],
                         'num_leaves': [15, 20, 31, 40],
                         'scale_pos_weight': [1.0, 0.8, 0.7]},
             scoring='roc_auc', verbose=1)

In [42]:
grid.best_params_

{'boosting_type': 'goss',
 'learning_rate': 0.05,
 'n_estimators': 1000,
 'num_leaves': 20,
 'scale_pos_weight': 0.7}

In [54]:
lgbm = LGBMClassifier(boosting_type = 'goss', learning_rate = 0.05,n_estimators = 1000,num_leaves = 20, scale_pos_weight = 0.7)
lgbm.fit(X_train,Y_train)
predictions_lgbm = lgbm.predict(X_test)
# Use accuracy_score function to get the accuracy
print("ligbm Accuracy Score -> ",accuracy_score(predictions_lgbm, Y_test)*100)
print(classification_report(Y_test,predictions_lgbm))

ligbm Accuracy Score ->  87.2340425531915
              precision    recall  f1-score   support

          -1       0.86      0.81      0.84        75
           1       0.88      0.91      0.90       113

    accuracy                           0.87       188
   macro avg       0.87      0.86      0.87       188
weighted avg       0.87      0.87      0.87       188



In [46]:
rf = RandomForestClassifier(n_estimators=1000,min_samples_leaf=2,
                                min_samples_split= 10, max_features = 'sqrt', criterion = 'entropy', bootstrap= True,
                              random_state=42)
rf.fit(X_train,Y_train) 
predictions_rf = rf.predict(X_test)
# Use accuracy_score function to get the accuracy
print("rf Accuracy Score -> ",accuracy_score(predictions_rf, Y_test)*100)
print(classification_report(Y_test,predictions_rf))

rf Accuracy Score ->  87.2340425531915
              precision    recall  f1-score   support

          -1       0.89      0.77      0.83        75
           1       0.86      0.94      0.90       113

    accuracy                           0.87       188
   macro avg       0.88      0.86      0.86       188
weighted avg       0.87      0.87      0.87       188

